In [1]:
import torch
import torchvision
from torch.autograd import Variable as V
from utils.visualise import make_dot

In [2]:
resnet_18 = torchvision.models.resnet18(pretrained=True)
resnet_18.eval();

In [3]:
# by setting the volatile flag to True, intermediate caches are not saved
# making the inspection of the graph pretty boring / useless
torch.manual_seed(0)
x = V(torch.randn(1, 3, 224, 224))#, volatile=True)
h_x = resnet_18(x)

In [4]:
dot = make_dot(h_x)  # generate network graph
dot.render('net.dot');  # save DOT and PDF in the current directory
# dot  # uncomment for displaying the graph in the notebook

In [5]:
# explore network graph
print('h_x creator ->',h_x.creator)
print('h_x creator prev fun type ->', type(h_x.creator.previous_functions))
print('h_x creator prev fun length ->', len(h_x.creator.previous_functions))
print('\n--- content of h_x creator prev fun ---')
for a, b in enumerate(h_x.creator.previous_functions): print(a, '-->', b)
print('---------------------------------------\n')

h_x creator -> <torch.nn._functions.linear.Linear object at 0x10cf46508>
h_x creator prev fun type -> <class 'tuple'>
h_x creator prev fun length -> 3

--- content of h_x creator prev fun ---
0 --> (<torch.autograd._functions.tensor.View object at 0x10cf46470>, 0)
1 --> (Parameter containing:
-1.8474e-02 -7.0461e-02 -5.1772e-02  ...  -3.9030e-02  1.7351e-01 -4.0976e-02
-8.1792e-02 -9.4370e-02  1.7355e-02  ...   2.0284e-01 -2.4782e-02  3.7172e-02
-3.3164e-02 -5.6569e-02 -2.4165e-02  ...  -3.4402e-02 -2.2659e-02  1.9705e-02
                ...                   ⋱                   ...                
-1.0300e-02  3.2804e-03 -3.5863e-02  ...  -2.7923e-02 -1.1458e-02  1.2759e-02
-3.5879e-02 -3.5296e-02 -2.9602e-02  ...  -3.2961e-02 -1.1022e-02 -5.1256e-02
 2.1277e-03 -2.4839e-02 -8.2920e-02  ...   4.1731e-02 -5.0030e-02  6.6327e-02
[torch.FloatTensor of size 1000x512]
, 0)
2 --> (Parameter containing:
1.00000e-02 *
 -0.2634
  0.3000
  0.0656
    ⋮   
 -1.7868
 -0.0782
 -0.6345
[torch.Float

The current node is a `torch.nn._functions.linear.Linear` object, fed by

- 0 --> output of `torch.autograd._functions.tensor.View` object
- 1 --> weight matrix of size `(1000, 512)`
- 2 --> bias vector of size `(1000)`

In [6]:
print(resnet_18)

ResNet (
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
  (relu): ReLU (inplace)
  (maxpool): MaxPool2d (size=(3, 3), stride=(2, 2), padding=(1, 1), dilation=(1, 1))
  (layer1): Sequential (
    (0): BasicBlock (
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
      (relu): ReLU (inplace)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
    )
    (1): BasicBlock (
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
      (relu): ReLU (inplace)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d

In [7]:
resnet_18._modules.keys()

odict_keys(['conv1', 'bn1', 'relu', 'maxpool', 'layer1', 'layer2', 'layer3', 'layer4', 'avgpool', 'fc'])

In [8]:
avgpool_layer = resnet_18._modules.get('avgpool')
h = avgpool_layer.register_forward_hook(
        lambda m, i, o: \
        print(
            'm:', type(m),
            '\ni:', type(i),
                '\n   len:', len(i),
                '\n   type:', type(i[0]),
                '\n   data size:', i[0].data.size(),
                '\n   data type:', i[0].data.type(),
            '\no:', type(o),
                '\n   data size:', o.data.size(),
                '\n   data type:', o.data.type(),
        )
)
h_x = resnet_18(x)
h.remove()

m: <class 'torch.nn.modules.pooling.AvgPool2d'> 
i: <class 'tuple'> 
   len: 1 
   type: <class 'torch.autograd.variable.Variable'> 
   data size: torch.Size([1, 512, 7, 7]) 
   data type: torch.FloatTensor 
o: <class 'torch.autograd.variable.Variable'> 
   data size: torch.Size([1, 512, 1, 1]) 
   data type: torch.FloatTensor


In [9]:
my_embedding = torch.zeros(512)
def fun(m, i, o): my_embedding.copy_(o.data)
h = avgpool_layer.register_forward_hook(fun)
h_x = resnet_18(x)
h.remove()

In [10]:
# print first values of the embedding
my_embedding[:10].view(1, -1)


 0.3879  0.0205  0.0268  2.9453  0.0234  0.0000  0.0000  0.7327  1.0997  0.0000
[torch.FloatTensor of size 1x10]